# Milestone 2
 * Install python mongodb connectors for connecting with Pyspark
    * !pip3 install pymongo[srv]
    * !pip3 install Pyspark
    * !pip3 install mlflow

* Load the data from mongodb
* Perform EDA to study the data
* Select the relevant columns for modeling
* Split the data for train and test 
* Model the data with different algorithms
* Evaluate the models and select the best performing and optimized model
* Save the model using pyfunc/mlflow 

# Data Loading

In [25]:
import pandas as pd

In [26]:
df = pd.read_csv('news.csv')

In [27]:
df.head()

,title,summary,link,pub_date,topic
0,Elon Musk: The THREE questions investors shoul...,"Elon Musk, the billionaire CEO of electric car...",https://www.express.co.uk/finance/city/1503278...,08-10-2021 23:09,finance
1,"Elon Musk Posts Puppy Floki In A Tesla, SHIB C...",A Twitter post of Elon Musk's Shiba Inu dog ha...,https://www.ibtimes.com/elon-musk-posts-puppy-...,04-10-2021 17:37,news
2,Cryptocurrency named after Elon Musk's dog sur...,Fans of Elon Musk made a cryptocurrency named ...,https://nypost.com/2021/10/06/cryptocurrency-n...,06-10-2021 15:28,news
3,Elon Musk says Tesla moving headquarters to Texas,Tesla chief Elon Musk says the company is movi...,https://www.news.com.au/breaking-news/elon-mus...,08-10-2021 02:23,news
4,Grimes Trolls Paparazzi with Communist Manifes...,"Grimes, Elon Musks Ex was spotted on the stree...",https://www.yahoo.com/entertainment/grimes-tro...,03-10-2021 22:29,entertainment


In [28]:
df['topic'].count()

25

In [29]:
df['topic'].value_counts()

news             18
tech              2
business          2
finance           1
science           1
entertainment     1
Name: topic, dtype: int64

In [6]:
pip install pyspark

     |████████████████████████████████| 212.4 MB 65 kB/s 
     |████████████████████████████████| 198 kB 47.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=4096e1b791805551942b5117e36737f3d24a7d137652673d5c5182aec01ddcbe
  Stored in directory: /root/.cache/pip/wheels/a5/0a/c1/9561f6fecb759579a7d863dcd846daaa95f598744e71b02c77
Successfully built pyspark


In [8]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local')
spark = SparkSession(sc)

In [30]:
# Pandas to Spark
train_df = spark.createDataFrame(df)

In [19]:
df['label'] = df['topic']

In [ ]:
df = df.drop('label')

In [31]:
train_df.columns

['title', 'summary', 'link', 'pub_date', 'topic']

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer
tokenizer = Tokenizer(inputCol="summary", outputCol="words")
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")
lr = LogisticRegression(maxIter=10, regParam=0.001)
pipeline = Pipeline(stages=[tokenizer, hashingTF, lr])
model = pipeline.fit(train_df)
mlflow.spark.log_model(model, "spark-model")

#Model Pipeline
regexTokenizer: Tokenization (with Regular Expression)
stopwordsRemover: Remove Stop Words
countVectors: Count vectors (“document-term vectors”)

In [32]:
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer
from pyspark.ml.classification import LogisticRegression
# regular expression tokenizer
regexTokenizer = RegexTokenizer(inputCol="summary", outputCol="words", pattern="\\W")
# stop words
add_stopwords = ["http","https","amp","rt","t","c","the"] 
stopwordsRemover = StopWordsRemover(inputCol="words", outputCol="filtered").setStopWords(add_stopwords)
# bag of words count
countVectors = CountVectorizer(inputCol="filtered", outputCol="features", vocabSize=10000, minDF=5)

In [33]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
label_stringIdx = StringIndexer(inputCol = "topic", outputCol = "label")
pipeline = Pipeline(stages=[regexTokenizer, stopwordsRemover, countVectors, label_stringIdx])
# Fit the pipeline to training documents.
pipelineFit = pipeline.fit(train_df)
dataset = pipelineFit.transform(train_df)
dataset.show(5)

+--------------------+--------------------+--------------------+----------------+-------------+--------------------+--------------------+--------------------+-----+
|               title|             summary|                link|        pub_date|        topic|               words|            filtered|            features|label|
+--------------------+--------------------+--------------------+----------------+-------------+--------------------+--------------------+--------------------+-----+
|Elon Musk: The TH...|Elon Musk, the bi...|https://www.expre...|08-10-2021 23:09|      finance|[elon, musk, the,...|[elon, musk, bill...|(67,[0,1,2,3,5,6,...|  4.0|
|Elon Musk Posts P...|A Twitter post of...|https://www.ibtim...|04-10-2021 17:37|         news|[a, twitter, post...|[a, twitter, post...|(67,[0,1,2,3,4,5,...|  0.0|
|Cryptocurrency na...|Fans of Elon Musk...|https://nypost.co...|06-10-2021 15:28|         news|[fans, of, elon, ...|[fans, of, elon, ...|(67,[0,1,2,3,4,5,...|  0.0|
|Elon Musk

In [34]:
# set seed for reproducibility
(trainingData, testData) = dataset.randomSplit([0.7, 0.3], seed = 100)
print("Training Dataset Count: " + str(trainingData.count()))
print("Test Dataset Count: " + str(testData.count()))

Training Dataset Count: 18
Test Dataset Count: 7


In [35]:
lr = LogisticRegression(maxIter=20, regParam=0.3, elasticNetParam=0)
lrModel = lr.fit(trainingData)
predictions = lrModel.transform(testData)
predictions.filter(predictions['prediction'] == 0) \
    .select("summary","topic","probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 10, truncate = 30)

+------------------------------+--------+------------------------------+-----+----------+
|                       summary|   topic|                   probability|label|prediction|
+------------------------------+--------+------------------------------+-----+----------+
|The era of dog-related cryp...|    news|[0.9354890025075915,0.00786...|  0.0|       0.0|
|Elon Musk, the billionaire ...| finance|[0.906509831922795,0.016388...|  4.0|       0.0|
|Grimes said she 'trolled' p...|business|[0.8462419435483275,0.00517...|  1.0|       0.0|
|Grimes and Tesla CEO Elon M...|    news|[0.7936617355502473,0.01619...|  0.0|       0.0|
|These truly weird photos of...|    news|[0.769110918417554,0.020765...|  0.0|       0.0|
|After making electric cars ...|    news|[0.6811122539162373,0.02365...|  0.0|       0.0|
+------------------------------+--------+------------------------------+-----+----------+



In [36]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(predictions)

0.5194805194805194

#Logistic Regression using TF-IDF Features

In [38]:
from pyspark.ml.feature import HashingTF, IDF
hashingTF = HashingTF(inputCol="filtered", outputCol="rawFeatures", numFeatures=10000)
idf = IDF(inputCol="rawFeatures", outputCol="features", minDocFreq=5) #minDocFreq: remove sparse terms
pipeline = Pipeline(stages=[regexTokenizer, stopwordsRemover, hashingTF, idf, label_stringIdx])
pipelineFit = pipeline.fit(train_df)
dataset = pipelineFit.transform(train_df)
(trainingData, testData) = dataset.randomSplit([0.7, 0.3], seed = 100)
lr = LogisticRegression(maxIter=20, regParam=0.3, elasticNetParam=0)
lrModel = lr.fit(trainingData)
predictions = lrModel.transform(testData)
predictions.filter(predictions['prediction'] == 0) \
    .select("summary","topic","probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 10, truncate = 30)

+------------------------------+--------+------------------------------+-----+----------+
|                       summary|   topic|                   probability|label|prediction|
+------------------------------+--------+------------------------------+-----+----------+
|The era of dog-related cryp...|    news|[0.9494596589517151,0.00554...|  0.0|       0.0|
|Elon Musk, the billionaire ...| finance|[0.8906003112018421,0.01642...|  4.0|       0.0|
|Grimes said she 'trolled' p...|business|[0.8350511863500334,0.00531...|  1.0|       0.0|
|Grimes and Tesla CEO Elon M...|    news|[0.7776767263527842,0.01514...|  0.0|       0.0|
|After making electric cars ...|    news|[0.7251769168395397,0.01917...|  0.0|       0.0|
|These truly weird photos of...|    news|[0.6604150825002939,0.02026...|  0.0|       0.0|
+------------------------------+--------+------------------------------+-----+----------+



In [39]:
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(predictions)

0.5194805194805194

#Cross-Validation

In [40]:
pipeline = Pipeline(stages=[regexTokenizer, stopwordsRemover, countVectors, label_stringIdx])
pipelineFit = pipeline.fit(train_df)
dataset = pipelineFit.transform(train_df)
(trainingData, testData) = dataset.randomSplit([0.7, 0.3], seed = 100)
lr = LogisticRegression(maxIter=20, regParam=0.3, elasticNetParam=0)
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
# Create ParamGrid for Cross Validation
paramGrid = (ParamGridBuilder()
             .addGrid(lr.regParam, [0.1, 0.3, 0.5]) # regularization parameter
             .addGrid(lr.elasticNetParam, [0.0, 0.1, 0.2]) # Elastic Net Parameter (Ridge = 0)
#            .addGrid(model.maxIter, [10, 20, 50]) #Number of iterations
#            .addGrid(idf.numFeatures, [10, 100, 1000]) # Number of features
             .build())
# Create 5-fold CrossValidator
cv = CrossValidator(estimator=lr, \
                    estimatorParamMaps=paramGrid, \
                    evaluator=evaluator, \
                    numFolds=5)
cvModel = cv.fit(trainingData)

predictions = cvModel.transform(testData)
# Evaluate best model
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(predictions)

0.5952380952380951

#Naive bayes

In [ ]:
from pyspark.ml.classification import NaiveBayes
nb = NaiveBayes(smoothing=1)
model = nb.fit(trainingData)
predictions = model.transform(testData)
predictions.filter(predictions['prediction'] == 0) \
    .select("summary","topic","probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 10, truncate = 30)

+------------------------------+-------+------------------------------+-----+----------+
|                       summary|  topic|                   probability|label|prediction|
+------------------------------+-------+------------------------------+-----+----------+
|The era of dog-related cryp...|   news|[0.9999896012069359,9.52542...|  0.0|       0.0|
|Elon Musk, the billionaire ...|finance|[0.9975052807878101,2.77499...|  4.0|       0.0|
|Grimes and Tesla CEO Elon M...|   news|[0.8236860597259401,7.25249...|  0.0|       0.0|
+------------------------------+-------+------------------------------+-----+----------+



In [ ]:
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(predictions)

0.35714285714285715

#Random Forest

In [ ]:
from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier(labelCol="label", \
                            featuresCol="features", \
                            numTrees = 100, \
                            maxDepth = 4, \
                            maxBins = 32)
# Train model with Training Data
rfModel = rf.fit(trainingData)
predictions = rfModel.transform(testData)
predictions.filter(predictions['prediction'] == 0) \
    .select("summary","topic","probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 10, truncate = 30)

+------------------------------+--------+------------------------------+-----+----------+
|                       summary|   topic|                   probability|label|prediction|
+------------------------------+--------+------------------------------+-----+----------+
|Elon Musk, the billionaire ...| finance|[0.8975164835164835,0.01066...|  4.0|       0.0|
|The era of dog-related cryp...|    news|[0.8512664835164835,0.02066...|  0.0|       0.0|
|Grimes said she 'trolled' p...|business|[0.7022857142857142,0.00500...|  1.0|       0.0|
|These truly weird photos of...|    news|[0.6731831501831502,0.05066...|  0.0|       0.0|
|After making electric cars ...|    news|[0.6622664835164837,0.04066...|  0.0|       0.0|
|Grimes and Tesla CEO Elon M...|    news|[0.6408498168498169,0.02,0....|  0.0|       0.0|
|Photo: Getty ImagesPhoto: G...|    news|[0.47653571428571423,0.02,0...|  0.0|       0.0|
+------------------------------+--------+------------------------------+-----+----------+



In [ ]:
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(predictions)

0.5952380952380951

In [37]:
import pickle

#Pickle implementation for Linear Regression
pickle.dump(lrModel, open('lr_model.pkl','wb'))
# Loading model to compare the results
model = pickle.load(open('lr_model.pkl','rb'))
print(model)

TypeError: ignored